In [ ]:
%pip install -U weaviate-client

In [ ]:
import os
import weaviate
from weaviate import classes as wvc

# aqui vamos usar a versão embedded, para testes/desenvolvimento, que roda direto do cliente
# o recomendado para produção é docker ou kubernetes
client = weaviate.connect_to_embedded(
    version="1.26.7",
    headers={
        "X-Cohere-Api-Key": os.environ.get("COHERE_APIKEY")
    }
)
# vamos mostrar a versão do cliente e do servidor
print(weaviate.__version__, client.get_meta().get("version"))

In [2]:
# vamos garantir que não temos essa coleção
client.collections.delete("Animais")
# e criar uma usando os modulos de vetorizador e gerador do cohere
collection = client.collections.create(
    "Animais",
    vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_cohere(),
    generative_config=wvc.config.Configure.Generative.cohere()
)
# agora vamos ingerir alguns dados.
# se precisar ingerir muitos dados, 
# use o batch ao invés do insert_many
collection.data.insert_many([
    {"text": "Leão é o rei da floresta"},
    {"text": "Gatos são fofos, mas estranhos"},
    {"text": "A girafa, calada, lá de cima vê tudo."},
    {"text": "Cachorros são fofos"},
])

{"build_git_commit":"fdb81e790","build_go_version":"go1.23.2","build_image_tag":"localhost","build_wv_version":"1.26.7","level":"warning","msg":"prop len tracker file /Users/dudanogueira/.local/share/weaviate/animais/1CJSI99dKNtr/proplengths does not exist, creating new tracker","time":"2024-10-21T10:02:50-03:00"}
{"action":"hnsw_prefill_cache_async","build_git_commit":"fdb81e790","build_go_version":"go1.23.2","build_image_tag":"localhost","build_wv_version":"1.26.7","level":"info","msg":"not waiting for vector cache prefill, running in background","time":"2024-10-21T10:02:50-03:00","wait_for_cache_prefill":false}
{"build_git_commit":"fdb81e790","build_go_version":"go1.23.2","build_image_tag":"localhost","build_wv_version":"1.26.7","level":"info","msg":"Created shard animais_1CJSI99dKNtr in 1.674833ms","time":"2024-10-21T10:02:50-03:00"}
{"action":"hnsw_vector_cache_prefill","build_git_commit":"fdb81e790","build_go_version":"go1.23.2","build_image_tag":"localhost","build_wv_version":"1

BatchObjectReturn(_all_responses=[UUID('6baee136-4e5b-446c-a431-d5ea343c1054'), UUID('e97d6e8f-7756-430d-a416-2bc3f1d50a46'), UUID('8def24bc-34db-4677-9fcc-4bced5a2bbdd'), UUID('d1418c33-12f6-46c9-b222-ef682dd8f183')], elapsed_seconds=0.38548803329467773, errors={}, uuids={0: UUID('6baee136-4e5b-446c-a431-d5ea343c1054'), 1: UUID('e97d6e8f-7756-430d-a416-2bc3f1d50a46'), 2: UUID('8def24bc-34db-4677-9fcc-4bced5a2bbdd'), 3: UUID('d1418c33-12f6-46c9-b222-ef682dd8f183')}, has_errors=False)

In [3]:
# a qualquer momento você pode "pegar" uma coleção
collection = client.collections.get("Animais")
collection.aggregate.over_all()

AggregateReturn(properties={}, total_count=4)

In [4]:
query = "家養動物"
#query = "animais domésticos"
#query = "domestic animals"
print("Query", query)
for obj in collection.query.near_text(
    query,
    return_metadata=wvc.query.MetadataQuery(distance=True)
).objects:
    print(f"\n### distância {obj.metadata.distance}")
    print(obj.properties)

Query 家養動物

### distância 0.49532848596572876
{'text': 'Cachorros são fofos'}

### distância 0.5232204794883728
{'text': 'Gatos são fofos, mas estranhos'}

### distância 0.5345996618270874
{'text': 'A girafa, calada, lá de cima vê tudo.'}

### distância 0.539834201335907
{'text': 'Leão é o rei da floresta'}


In [5]:
# O hybrid search vai "fundir" a distância dos vetores
# junto do score da busca bm25. Agora não temos mais distância, e sim score
# que você pode ver explicado no explain_score

# adicionamos também o auto_limit, que limita os objetos por grupos de proximidade. 
# com isso exibiremos somente o grupo 1. 
# Quando há um salto maior no score, se agrupa no grupo 2, etc.
for obj in collection.query.hybrid(
    "animais selvagens de safari africano para turista",
    return_metadata=wvc.query.MetadataQuery(score=True, explain_score=True),
    auto_limit=1
).objects:
    print(f"\n ### score: {obj.metadata.score}")
    print(f"explain score: {obj.metadata.explain_score}")
    print(obj.properties)


 ### score: 0.8646168112754822
explain score: 
Hybrid (Result Set keyword,bm25) Document 8def24bc-34db-4677-9fcc-4bced5a2bbdd: original score 0.461453, normalized score: 0.3 - 
Hybrid (Result Set vector,hybridVector) Document 8def24bc-34db-4677-9fcc-4bced5a2bbdd: original score 0.53033376, normalized score: 0.5646168
{'text': 'A girafa, calada, lá de cima vê tudo.'}

 ### score: 0.699999988079071
explain score: 
Hybrid (Result Set vector,hybridVector) Document 6baee136-4e5b-446c-a431-d5ea343c1054: original score 0.5343678, normalized score: 0.7
{'text': 'Leão é o rei da floresta'}


In [6]:
# agora podemos fazer ações para cada objeto (single_prompt)
query = collection.generate.hybrid(
    "animais selvagens de safari africano para turista",
    return_metadata=wvc.query.MetadataQuery(score=True, explain_score=True),
    auto_limit=1,
    single_prompt="Gere uma piada sobre: {text}",
)
for obj in query.objects:
    print(f"\n ### score: {obj.metadata.score}")
    print(f"explain score: {obj.metadata.explain_score}")
    print(obj.properties)
    print(f"Texto gerado: {obj.generated}")


 ### score: 0.8646168112754822
explain score: 
Hybrid (Result Set keyword,bm25) Document 8def24bc-34db-4677-9fcc-4bced5a2bbdd: original score 0.461453, normalized score: 0.3 - 
Hybrid (Result Set vector,hybridVector) Document 8def24bc-34db-4677-9fcc-4bced5a2bbdd: original score 0.53033376, normalized score: 0.5646168
{'text': 'A girafa, calada, lá de cima vê tudo.'}
Texto gerado: Por que a girafa é a melhor espiã da savana? Porque calada, lá de cima, ela vê tudo!

 ### score: 0.699999988079071
explain score: 
Hybrid (Result Set vector,hybridVector) Document 6baee136-4e5b-446c-a431-d5ea343c1054: original score 0.5343678, normalized score: 0.7
{'text': 'Leão é o rei da floresta'}
Texto gerado: Por que o leão é considerado o rei da floresta? Porque ele é o único que consegue rugir sem precisar de um microfone!


In [9]:
# E também ações para o grupo de objetos encontrados (grouped_task)
# agora podemos fazer ações para cada objeto (singleTask)
query = collection.generate.hybrid(
    "animais selvagens de safari africano para turista",
    return_metadata=wvc.query.MetadataQuery(score=True, explain_score=True),
    auto_limit=1,
    single_prompt="Gere uma piada sobre: {text}",
    grouped_task="Gere uma piada sobre os textos fornecidos, utilize algo sobre inteligência artificial",
)
for obj in query.objects:
    print(f"\n ### score: {obj.metadata.score}")
    print(f"explain score: {obj.metadata.explain_score}")
    print(obj.properties)
    print(f"Texto gerado: {obj.generated}")

print(f"\n\nConteúdo gerado em conjunto: {query.generated}")


 ### score: 0.8646168112754822
explain score: 
Hybrid (Result Set keyword,bm25) Document 8def24bc-34db-4677-9fcc-4bced5a2bbdd: original score 0.461453, normalized score: 0.3 - 
Hybrid (Result Set vector,hybridVector) Document 8def24bc-34db-4677-9fcc-4bced5a2bbdd: original score 0.53033376, normalized score: 0.5646168
{'text': 'A girafa, calada, lá de cima vê tudo.'}
Texto gerado: Por que a girafa é a melhor espiã da savana? Porque calada, lá de cima, ela vê tudo!

 ### score: 0.699999988079071
explain score: 
Hybrid (Result Set vector,hybridVector) Document 6baee136-4e5b-446c-a431-d5ea343c1054: original score 0.5343678, normalized score: 0.7
{'text': 'Leão é o rei da floresta'}
Texto gerado: Por que o leão é considerado o rei da floresta? Porque ele é o único que consegue rugir sem precisar de um microfone!


Conteúdo gerado em conjunto: Por que a inteligência artificial não gosta de piadas sobre animais? Porque ela prefere piadas mais "tecnológicas", como: "A girafa, calada, lá de ci

In [10]:
# Pronto! Agora vamos fechar o cliente para desligar o servidor que está rodando
client.close()

{"action":"restapi_management","build_git_commit":"fdb81e790","build_go_version":"go1.23.2","build_image_tag":"localhost","build_wv_version":"1.26.7","docker_image_tag":"localhost","level":"info","msg":"Shutting down... ","time":"2024-10-21T10:06:12-03:00"}
{"action":"restapi_management","build_git_commit":"fdb81e790","build_go_version":"go1.23.2","build_image_tag":"localhost","build_wv_version":"1.26.7","docker_image_tag":"localhost","level":"info","msg":"Stopped serving weaviate at http://127.0.0.1:8079","time":"2024-10-21T10:06:12-03:00"}
{"action":"telemetry_push","build_git_commit":"fdb81e790","build_go_version":"go1.23.2","build_image_tag":"localhost","build_wv_version":"1.26.7","level":"info","msg":"telemetry terminated","payload":"\u0026{MachineID:439c7d8b-afe6-4776-9ee9-047de5c0f9f6 Type:TERMINATE Version:1.26.7 NumObjects:14 OS:darwin Arch:arm64 UsedModules:[generative-cohere text2vec-cohere]}","time":"2024-10-21T10:06:13-03:00"}
{"build_git_commit":"fdb81e790","build_go_vers